In [ ]:
import time
import logging
import pandas as pd
import requests

In [ ]:
logging.basicConfig(
    level=logging.ERROR,
    filename='app.log',
    filemode='a',
    format='%(message)s'
)

def create_url(
        country_code="2060",
        station_code="00000",
        day_start="01",
        day_end="31",
        month="01",
        year="2000",
        page="1"
):
    url = f"http://www.meteomanz.com/sy2?cou={country_code}&ty=hp&ind={station_code}&d1={day_start}&m1={month}&y1={year}&d2={day_end}&m2={month}&y2={year}&so=001&np={page}"    
    return url


def create_header(
        userAgent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        referer="http://www.meteomanz.com/",
        host="www.meteomanz.com",
        acceptLanguage="en-US,en;q=0.9",
        connection="keep-alive",
        accept="text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"
):
    headers = {
        "User-Agent": userAgent,
        "Referer": referer,
        "Host": host,
        "Accept-Language": acceptLanguage,
        "Connection": connection,
        "Accept": accept
    }
    return headers

YEARS = [*map(str, range(2000, 2025, 1))]
MONTH = [*map(lambda x: str(x).zfill(2), range(1, 13, 1))]
DAY = [*map(lambda x: str(x).zfill(2), range(1, 32, 1))]
PAGE = [*map(str, range(1, 13, 1))]

In [ ]:
HEADERS = create_header()

for y in YEARS:
    data = pd.DataFrame()
    for m in MONTH:
        for p in PAGE:
            URL = create_url(
                country_code="2060",
                station_code="00000",
                day_start="01",
                day_end="31",
                month=m,
                year=y,
                page=p
            )
            # r = requests.get(url = URL, headers = HEADERS)
            # while r.status_code != 200:
            #     time.sleep(3)
            #     logging.error(f"Status Code: {r.status_code} - {y}-{m}:{p}")
            #     r = requests.get(url = URL, headers = HEADERS)
            get_data = True
            while get_data:
                try:
                    df = pd.read_html(URL, storage_options=HEADERS)[0]
                    print(f"{y}-{m}:{p}({len(df)})")
                    if len(df) != 0:
                        data = pd.concat([data, df])
                        data.reset_index(drop=True, inplace=True)
                    get_data = False
                except:
                    time.sleep(3)
                    r = requests.get(url = URL, headers = HEADERS)
                    logging.error(f"Status Code: {r.status_code} - {y}-{m}:{p}")
                    get_data = True
                    
                    
                    
                    
                # try:
                #     df = pd.read_html(URL, storage_options=HEADERS)[0]
                #     if len(df) != 0:
                #         data = pd.concat([data, df])
                #         data.reset_index(drop=True, inplace=True)
                # except:
                #     time.sleep(3)
                #     try:
                #         df = pd.read_html(URL, storage_options=HEADERS)[0]
                #         if len(df) != 0:
                #             data = pd.concat([data, df])
                #             data.reset_index(drop=True, inplace=True)
                #     except:
                #         logging.error(f"Error in {y}-{m}:{p}")
                #         print(f"Error in {y}-{m}:{p}")
                #         continue
    data.to_csv(f"output/{y}.csv", index=False)
